In [1]:
#..MODIFIED BY CHUYAN at Nov 12th, this file was intended to test for a procedure of read various GCM dataset and automatically analysis 
#..their behaviors of how LWP(TWP) amount changes with CCFs and to build a (linear) regression model to predict these change;
#..among these process, we wish to examine a way to automatically distinguish these LRMs and determine their threshold;



import netCDF4
from numpy import *
import matplotlib.pyplot as plt
import xarray as xr
import PyNIO as Nio
import pandas as pd

from scipy.stats import *
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import BoundaryNorm

from area_mean import *
from binned_cyFunctions5 import *
from read_hs_file import read_var_mod


from get_LWPCMIP6 import *

In [3]:

def get_LWPCMIP6(modn='IPSL-CM6A-LR', consort='IPSL', cmip='cmip6', exper='', ensmem='r1i1p1f1', gg='gr', typevar='Amon'):
    #if exper == 'historical':
    #    amip = False
    #else:
    #    amip = True
    #print(modn)
#   modn='IPSL-CM5A-LR'; consort='IPSL'; cmip='cmip5'; exper='amip'; ensmem='r1i1p1'; gg=''; typevar='Amon'
    if exper == 'historical':
        time1 = [1950, 1, 1]
        time2 = [2014, 12, 30]
    if exper == 'amip':
        time1 = [1950, 1, 1]
        time2 = [2007, 12, 30]
    
    
    
    #..abrupt4xCO2
    exper = 'abrupt-4xCO2'
    if exper == 'abrupt-4xCO2':
        
        TEST1_time= read_var_mod(modn=modn,consort=consort,varnm='tas',cmip=cmip,exper=exper,ensmem=ensmem,gg=gg,typevar=typevar,time1=[1,1,1],time2=[2149,12,31])[-1]
        time1=[int(min(TEST1_time[:,0])),1,1]
        time2=[int(max(TEST1_time[:,0])),12,31]
        
        print(time1, time2)
        
    
    sfc_T_abr       = read_var_mod(modn=modn, consort=consort, varnm='ts', cmip=cmip, exper= exper, ensmem=ensmem, typevar=typevar, gg=gg, read_p=False, time1= time1, time2= time2)[0]

    T_700_alevs_abr,Pres_abr,lat_abr,lon_abr,times_abr = read_var_mod(modn=modn, consort=consort, varnm='ta', cmip=cmip, exper= exper, ensmem=ensmem, typevar=typevar, gg=gg, read_p=True, time1= time1, time2= time2)
    T_700_abr = T_700_alevs_abr[:, 3,:,:]   #..700 hPa levels

    sfc_P_abr       = read_var_mod(modn=modn, consort=consort, varnm='ps', cmip=cmip, exper= exper, ensmem=ensmem, typevar=typevar, gg=gg, read_p=False, time1= time1, time2= time2)[0]   

    sub_abr         = read_var_mod(modn=modn, consort=consort, varnm='wap', cmip=cmip, exper= exper, ensmem=ensmem, typevar=typevar, gg=gg, read_p=True, time1= time1, time2= time2)[0][:, 5,:,:]
    #..500mb downward motions

    clivi_abr       = read_var_mod(modn=modn, consort=consort, varnm='clivi', cmip=cmip, exper= exper, ensmem=ensmem, typevar=typevar, gg=gg, read_p=False, time1= time1, time2= time2)[0]
    clwvi_abr       = read_var_mod(modn=modn, consort=consort, varnm='clwvi', cmip=cmip, exper= exper, ensmem=ensmem, typevar=typevar, gg=gg, read_p=False, time1= time1, time2= time2)[0]
    
    tas_abr         = read_var_mod(modn=modn, consort=consort, varnm='tas', cmip=cmip, exper= exper, ensmem=ensmem, typevar=typevar, gg=gg, read_p=False, time1= time1, time2= time2)[0]

    P_abr           = read_var_mod(modn=modn, consort=consort, varnm='pr', cmip=cmip, exper= exper, ensmem=ensmem, typevar=typevar, gg=gg, read_p=False, time1= time1, time2= time2)[0]

    E_abr           = read_var_mod(modn=modn, consort=consort, varnm='evspsbl', cmip=cmip, exper= exper, ensmem=ensmem, typevar=typevar, gg=gg, read_p=False, time1= time1, time2= time2)[0]
    
    prw_abr         = read_var_mod(modn=modn, consort=consort, varnm='prw', cmip=cmip, exper= exper, ensmem=ensmem, typevar=typevar, gg=gg, read_p=False, time1= time1, time2= time2)[0]

    print(sfc_T_abr.shape)
    #..1800 months =150 yrs for CESM2:abrupt experiemnt    
    inputVar_abr = {'sfc_T': sfc_T_abr, 'T_700': T_700_abr, 'sfc_P': sfc_P_abr, 'sub': sub_abr, 'clivi': clivi_abr,
                 'clwvi': clwvi_abr, 'tas': tas_abr, 'P': P_abr, 'E': E_abr, 'prw': prw_abr, 'pres': Pres_abr, 'lat':lat_abr, 'lon':lon_abr, 'times':times_abr}
    
    
    
    #..pi-Control
    exper = 'piControl'

    if exper == 'piControl':
        TEST2_time= read_var_mod(modn=modn,consort=consort,varnm='tas',cmip=cmip,exper=exper,ensmem=ensmem,gg=gg,typevar=typevar,time1=[1,1,1], time2=[2349,12,31])[-1]
        timep1=[int(max(TEST2_time[:,0]))- 99,1,1]
        timep2=[int(max(TEST2_time[:,0])),12,31]
        
        print (timep1, timep2)
    
    sfc_T       = read_var_mod(modn='CESM2', consort='NCAR', varnm='ts', cmip='cmip6', exper= exper, ensmem=ensmem, typevar='Amon', gg=gg, read_p=False, time1= timep1, time2= timep2)[0]

    T_700_alevs,Pres_pi,lat_pi,lon_pi,times_pi =  read_var_mod(modn='CESM2', consort='NCAR', varnm='ta', cmip='cmip6', exper= exper, ensmem=ensmem, typevar='Amon', gg=gg, read_p=True, time1= timep1, time2= timep2)
    T_700       = T_700_alevs[:, 3,:,:]

    sfc_P       = read_var_mod(modn='CESM2', consort='NCAR', varnm='ps', cmip='cmip6', exper= exper, ensmem=ensmem, typevar='Amon', gg=gg, read_p=False, time1= timep1, time2= timep2)[0]   
    #..sea surface Pressure, Units in Pa

    sub         = read_var_mod(modn='CESM2', consort='NCAR', varnm='wap', cmip='cmip6', exper= exper, ensmem=ensmem, typevar='Amon', gg=gg, read_p=True, time1= timep1, time2= timep2)[0][:, 5,:,:]
    #..500mb downward motion 

    clivi       = read_var_mod(modn='CESM2', consort='NCAR', varnm='clivi', cmip='cmip6', exper= exper, ensmem=ensmem, typevar='Amon', gg=gg, read_p=False, time1= timep1, time2= timep2)[0]
    #..ICE WATER PATH, Units in kg m^-2
    clwvi       = read_var_mod(modn='CESM2', consort='NCAR', varnm='clwvi', cmip='cmip6', exper= exper, ensmem=ensmem, typevar='Amon', gg=gg, read_p=False, time1= timep1, time2= timep2)[0]
    
    tas         = read_var_mod(modn='CESM2', consort='NCAR', varnm='tas', cmip='cmip6', exper= exper, ensmem=ensmem, typevar='Amon', gg=gg, read_p=False, time1= timep1, time2= timep2)[0]
    #..2-m air Temperature, for 'gmt'

    P           = read_var_mod(modn='CESM2', consort='NCAR', varnm='pr', cmip='cmip6', exper= exper, ensmem=ensmem, typevar='Amon', gg=gg, read_p=False, time1= timep1, time2= timep2)[0]
    #..Precipitation, Units in kg m^-2 s^-1 = mm *s^-1
    E           = read_var_mod(modn='CESM2', consort='NCAR', varnm='evspsbl', cmip='cmip6', exper= exper, ensmem=ensmem, typevar='Amon', gg=gg, read_p=False, time1= timep1, time2= timep2)[0]
    #..Evaporations, Units also in kg m^-2 s^-1 = mm *s^-1
    
    prw_pi      =read_var_mod(modn='CESM2', consort='NCAR', varnm='prw', cmip='cmip6', exper= exper, ensmem=ensmem, typevar='Amon', gg=gg, read_p=False, time1= timep1, time2= timep2)[0]

    print(sfc_T.shape)
    #..6000 months =99 yrs for CESM2 piControl experiment
    
    inputVar_pi = {'sfc_T': sfc_T, 'T_700': T_700, 'sfc_P': sfc_P, 'sub': sub, 'clivi': clivi, 
                 'clwvi': clwvi, 'tas': tas, 'P': P, 'E': E, 'prw': prw_pi, 'pres':Pres_pi, 'lat':lat_pi, 'lon':lon_pi, 'times': times_pi}
    '''
    TS = read_var_mod(modn=modn,consort=consort,varnm='ts',cmip=cmip,exper=exper,ensmem=ensmem,gg=gg,typevar=typevar,time1=time1,time2=time2)
    '''
    
    #return {'SST0': TS[0], 'SST1': TSf[0], 'LWP0': LWP0, 'LWP1': LWP1, 'lat': IWP[2][:], 'lon': IWP[3][:], 'time0': IWP[-1], 'time1': IWPf[-1], 'IWP0': IWP[0], 'IWP1': IWPf[0], 'P0': P[0], 'E0': E[0], 'P1': Pf[0], 'E1': Ef[0], "WVP0": wvp[0], 'WVP1': wvpf[0], 'SI': SI, 'U100': U10_0, 'U101': U10_f, 'TAS0': TAS[0], 'TAS1': TASf[0], 'TA0': TA_0, 'TA1': TA_1, 'SW0': SW[0], 'SW0c': SWc[0], 'SW1': SWf[0], 'SW1c': SWfc[0],'TS0':TS[0],'TS1':TSf[0]}#,'WAP0':WAP_0,'WAP1':WAP_1,'LTS0':LTS,'LTS1':LTSf}
    return inputVar_pi, inputVar_abr


In [2]:

inputVar_pi, inputVar_abr = get_LWPCMIP6(modn='CESM2', consort='NCAR', cmip='cmip6', exper='', ensmem='r1i1p1f1', gg='gn', typevar='Amon')


['/glade/collections/cmip//CMIP6/CMIP/NCAR/CESM2/abrupt-4xCO2/r1i1p1f1/Amon/tas/gn/files/d20190828/', '/glade/collections/cmip//CMIP6/CMIP/NCAR/CESM2/abrupt-4xCO2/r1i1p1f1/Amon/tas/gn/files/d20190425/', '/glade/collections/cmip//CMIP6/CMIP/NCAR/CESM2/abrupt-4xCO2/r1i1p1f1/Amon/tas/gn/files/d20190927/']
/glade/collections/cmip//CMIP6/CMIP/NCAR/CESM2/abrupt-4xCO2/r1i1p1f1/Amon/tas/gn/files/d20190828//*tas**_**nc*
/glade/collections/cmip//CMIP6/CMIP/NCAR/CESM2/abrupt-4xCO2/r1i1p1f1/Amon/tas/gn/files/d20190828/tas_Amon_CESM2_abrupt-4xCO2_r1i1p1f1_gn_000101-015012.nc


/glade/work/chuyan/Research/linear_regression_CCFs_Clouds_metrics/read_hs_file.py:123: UserWarning: WARNING: missing_value not used since it
cannot be safely cast to variable data type
  data = f.variables[varnm][ind]


(1800, 192, 288)
[1, 1, 1] [150, 12, 31]
['/glade/collections/cmip//CMIP6/CMIP/NCAR/CESM2/abrupt-4xCO2/r1i1p1f1/Amon/ts/gn/files/d20190828/', '/glade/collections/cmip//CMIP6/CMIP/NCAR/CESM2/abrupt-4xCO2/r1i1p1f1/Amon/ts/gn/files/d20190425/', '/glade/collections/cmip//CMIP6/CMIP/NCAR/CESM2/abrupt-4xCO2/r1i1p1f1/Amon/ts/gn/files/d20190927/']
/glade/collections/cmip//CMIP6/CMIP/NCAR/CESM2/abrupt-4xCO2/r1i1p1f1/Amon/ts/gn/files/d20190828//*ts**_**nc*
/glade/collections/cmip//CMIP6/CMIP/NCAR/CESM2/abrupt-4xCO2/r1i1p1f1/Amon/ts/gn/files/d20190828/ts_Amon_CESM2_abrupt-4xCO2_r1i1p1f1_gn_000101-015012.nc
(1800, 192, 288)
['/glade/collections/cmip//CMIP6/CMIP/NCAR/CESM2/abrupt-4xCO2/r1i1p1f1/Amon/ta/gn/files/d20190828/', '/glade/collections/cmip//CMIP6/CMIP/NCAR/CESM2/abrupt-4xCO2/r1i1p1f1/Amon/ta/gn/files/d20190425/', '/glade/collections/cmip//CMIP6/CMIP/NCAR/CESM2/abrupt-4xCO2/r1i1p1f1/Amon/ta/gn/files/d20190927/']
/glade/collections/cmip//CMIP6/CMIP/NCAR/CESM2/abrupt-4xCO2/r1i1p1f1/Amon/ta/gn

In [ ]:
def do_stuff_2_data(dict_in):
    dict_in['varnew']=dict_in['oldvar1']*10+dict_in['oldvar2']
    

In [3]:
#..get the shapes of monthly data
shape_lat = len(inputVar_pi['lat'])
shape_lon = len(inputVar_pi['lon'])
shape_time_pi = len(inputVar_pi['times'])
shape_time_abr = len(inputVar_abr['times'])
#print(shape_lat, shape_lon, shape_time_pi, shape_time_abr)


#..choose lat 40~ 85°S as the Southern-Ocean Regions
lons        = inputVar_pi['lon']
lats        = inputVar_pi['lat'][:]

levels      = np.array(inputVar_abr['pres'])
times_abr   = inputVar_abr['times']
times_pi    = inputVar_pi['times']
#print(lats, lons)


lati0 = -40.
latsi0= min(range(len(lats)), key = lambda i: abs(lats[i] - lati0))
lati1 = -85.
latsi1= min(range(len(lats)), key = lambda i: abs(lats[i] - lati1))
print(latsi0, latsi1)

shape_latSO =  latsi0 - latsi1
#print(shape_latSO)


#..abrupt4xCO2 Variables: LWP, tas(gmt), SST, p-e
LWP_abr  = np.array(inputVar_abr['clwvi']) - np.array(inputVar_abr['clivi'])   #..units in kg m^-2

gmt_abr  = np.array(inputVar_abr['tas'])

SST_abr  = np.array(inputVar_abr['sfc_T'])


Precip_abr =  np.array(inputVar_abr['P']) * (24.*60.*60.)   #..Precipitation. Convert the units from kg m^-2 s^-1 -> mm*day^-1
#print(np.nanmean(Precip_abr))   #.. IPSL/abr2.80..  CNRM ESM2 1/abr 2.69.. CESM2/abr 2.74..
Eva_abr    =  np.array(inputVar_abr['E']) * (24.*60.*60.)   #..evaporation, mm day^-1
#print(np.nanmean(Eva_abr))         #.. IPSL/abr2.50..  CNRM ESM2 1/abr 2.43.. CESM2/abr 2.43..
MC_abr  = Precip_abr - Eva_abr   #..Moisture Convergence calculated from abrupt4xCO2's P - E, Units in mm day^-1

Twp_abr  = np.array(inputVar_abr['clwvi'])
Iwp_abr  = np.array(inputVar_abr['clivi'])
prw_abr  = np.array(inputVar_abr['prw'])

#print(Eva_abr.shape, ';the mean temperature of abr: ', np.nanmean(gmt_abr))

#..pi-Control Variables: LWP, tas(gmt), SST, p-e
LWP  = np.array(inputVar_pi['clwvi']) - np.array(inputVar_pi['clivi'])   #..units in kg m^-2
print(type(LWP))
gmt  = np.array(inputVar_pi['tas'])

SST  = np.array(inputVar_pi['sfc_T'])


Precip =  np.array(inputVar_pi['P'])* (24.*60.*60.)    #..Precipitation. Convert the units from kg m^-2 s^-1 -> mm*day^-1
#print(np.nanmean(Precip))   #.. IPSL/piC 2.43..CNRM/piC 2.40.. CESM2/PIc 2.39..
Eva    =  np.array(inputVar_pi['E']) * (24.*60.*60.)   #..evaporation, mm day^-1
#print(np.nanmean(Eva))   #.. IPSL/piC  2.21..CNRM/piC 2.20.. CESM2/PIc 2.17..

MC  = Precip - Eva   #..Moisture Convergence calculated from pi-Control's P - E, Units in mm day^-1

Twp  = np.array(inputVar_pi['clwvi'])
Iwp  = np.array(inputVar_pi['clivi'])
prw_pi  = np.array(inputVar_pi['prw'])

#print(Eva.shape, ';the mean temprature of PI-control: ', np.nanmean(gmt))

53 5
<class 'numpy.ndarray'>


In [35]:
#..abrupt4xCO2 
# Lower Tropospheric Stability:
k  = 0.286
theta_700_abr  = np.array(inputVar_abr['T_700']) * (100000./70000.)**k
theta_skin_abr = np.array(inputVar_abr['sfc_T']) * (100000./np.array(inputVar_abr['sfc_P']))**k 

LTS_m_abr  = theta_700_abr - theta_skin_abr


#..Subtract the outliers in T_700 and LTS_m, 'nan' comes from missing T_700 data
LTS_e_abr  = np.ma.masked_where(theta_700_abr >= 500, LTS_m_abr)


# Meteorology Subsidence at 500 hPa, units in Pa s^-1:
Subsidence_abr =  np.array(inputVar_abr['sub'])


#..pi-Control 
# Lower Tropospheric Stability:
theta_700  = np.array(inputVar_pi['T_700']) * (100000./70000.)**k
theta_skin = np.array(inputVar_pi['sfc_T']) * (100000./np.array(inputVar_pi['sfc_P']))**k 
LTS_m  = theta_700 - theta_skin

#..Subtract the outliers in T_700 and LTS_m 
LTS_e  = np.ma.masked_where(theta_700 >= 500, LTS_m)


#..Meteological Subsidence  at 500 hPa, units in Pa s^-1:
Subsidence =  np.array(inputVar_pi['sub'])


# define Dictionary to store: CCFs(4), gmt, other variables 
'''
GMT_abr   =  {'gmt': gmt_abr}
CCF_abr   =  {'SST': SST_abr, 'p_e': MC_abr, 'LTS': LTS_e_abr, 'SUB': Subsidence_abr}

GMT_pi   =  {'gmt': gmt}
CCF_pi   =  {'SST': SST, 'p_e': MC, 'LTS': LTS_e, 'SUB': Subsidence}

LWP_abr  =  {'LWP': LWP_abr, 'TWP': Twp_abr, 'IWP': Iwp_abr,  'PRW': prw_abr}
LWP_pi   =  {'LWP': LWP, 'TWP': Twp, 'IWP': Iwp,  'PRW': prw_pi}
'''

dict0_PI_var = {'gmt': gmt, 'LWP': LWP, 'TWP': Twp, 'IWP': Iwp,  'PRW': prw_pi, 'SST': SST, 'p_e': MC, 'LTS': LTS_e, 'SUB': Subsidence}

dict0_abr_var = {'gmt': gmt_abr, 'LWP': LWP_abr, 'TWP': Twp_abr, 'IWP': Iwp_abr,  'PRW': prw_abr, 'SST': SST_abr, 'p_e': MC_abr, 'LTS': LTS_e_abr, 'SUB': Subsidence_abr}


'''
dict_raw
data_vars=['lwp','tas',1,{'data':0}]
ann_means={}
for i in range(len(data_vars)):
    #dict_raw[data_vars[i]+'_ann']=np.nanmean(dict_raw[data_vars[i]])
    ann_mean[data_vars[i]]=np.nanmean(dict_raw[data_vars[i]])
    
dict_raw['ann']=ann_mean

savez('file',dict_raw)
'''



{'gmt': array([[[245.30347, 245.30347, 245.30347, ..., 245.25475, 245.27191,
         245.29791],
        [244.15715, 244.18431, 244.22488, ..., 244.08699, 244.10747,
         244.1288 ],
        [244.15611, 244.18762, 244.22954, ..., 244.10434, 244.11765,
         244.12805],
        ...,
        [240.19063, 240.19412, 240.19968, ..., 240.19096, 240.18471,
         240.18677],
        [240.39021, 240.39243, 240.39482, ..., 240.38516, 240.38664,
         240.38832],
        [240.57002, 240.56969, 240.56937, ..., 240.57129, 240.57083,
         240.5704 ]],

       [[236.26273, 236.26273, 236.26273, ..., 236.21422, 236.23135,
         236.25716],
        [235.01707, 235.0531 , 235.10146, ..., 234.90714, 234.94025,
         234.98305],
        [235.53651, 235.62111, 235.70284, ..., 235.36168, 235.41302,
         235.47066],
        ...,
        [241.42542, 241.36316, 241.3017 , ..., 241.57684, 241.53531,
         241.48172],
        [240.37126, 240.36842, 240.36699, ..., 240.38785, 240.38

In [36]:
# get the Annual-mean, Southern-Ocean region arrays

datavar_nas = ['gmt', 'LWP', 'TWP', 'IWP', 'PRW', 'SST','p_e','LTS','SUB' ]

dict1_PI_yr  = {}
dict1_abr_yr = {}

layover_yr_abr = np.zeros((shape_time_abr//12, shape_lat, shape_lon))
layover_yr_pi  = np.zeros((shape_time_pi//12, shape_lat, shape_lon))


a_array = dict0_abr_var['LWP']#dict0_abr_var[datavar_nas[a]]
print(a_array)

for a in range(len(datavar_nas)):
    a_array = dict0_abr_var[datavar_nas[a]]
    
    for i in range(shape_time_abr//12):
        #.. '//' representing 'int' division operation
        #layover_yr_abr[i,:,:] i*12:(i+1)*12
        #test=  nanmean(a_array[0:1,:,:], axis=0)
        test=a_array[0]
    dict1_abr_yr[datavar_nas[a]+'_yr'] =  layover_yr_abr
    
    
    b_array = dict0_PI_var[datavar_nas[a]]
    for j in range(shape_time_pi//12):
        layover_yr_pi[j,:,:]  = nanmean(b_array[j*12:(j+1)*12,:,:], axis=0)
    
    dict1_PI_yr[datavar_nas[a]+'_yr'] =  layover_yr_pi
        

#print(dict1_PI_yr['LWP_yr'], dict1_PI_yr['gmt_yr'].shape)




[[[0.0148282  0.01482821 0.01482822 ... 0.01482875 0.01482882 0.01482823]
  [0.01423743 0.01440555 0.01458182 ... 0.01391651 0.01399447 0.01409262]
  [0.01669285 0.0169216  0.01692128 ... 0.0162011  0.01634382 0.01654777]
  ...
  [0.01686475 0.01701647 0.01720829 ... 0.01622916 0.01646479 0.01664973]
  [0.01318416 0.01310323 0.01304733 ... 0.01347167 0.01336397 0.01328107]
  [0.0125244  0.01252444 0.01252447 ... 0.01252432 0.01252434 0.01252438]]

 [[0.00353787 0.00353787 0.00353787 ... 0.00353792 0.00353791 0.00353788]
  [0.00539959 0.00540161 0.00542856 ... 0.00543581 0.00541789 0.00540625]
  [0.00789335 0.00783093 0.00780272 ... 0.00827604 0.00815327 0.00802748]
  ...
  [0.01548493 0.01536706 0.01525709 ... 0.01600247 0.0157879  0.01559141]
  [0.00926443 0.00920099 0.0091655  ... 0.00948439 0.0094311  0.00933968]
  [0.00667177 0.00667173 0.00667174 ... 0.00667173 0.00667174 0.00667176]]

 [[0.00355396 0.0035539  0.00355387 ... 0.00355376 0.00355368 0.00355378]
  [0.00352198 0.003559

In [14]:
datavar_nas[0]


a_array = dict0_abr_var[datavar_nas[0]]
#print(a_array.shape, gmt_abr.shape)


#print(a_array, gmt_abr)

a_array

array([[[247.07126, 247.07126, 247.07126, ..., 247.02208, 247.03923,
         247.06549],
        [245.92903, 245.9711 , 246.02953, ..., 245.8211 , 245.85332,
         245.8898 ],
        [245.92001, 245.94833, 245.93799, ..., 245.86186, 245.87056,
         245.8839 ],
        ...,
        [238.98143, 238.9895 , 238.998  , ..., 238.97208, 238.97389,
         238.97728],
        [239.43126, 239.41821, 239.40443, ..., 239.46217, 239.45099,
         239.44081],
        [239.64412, 239.64334, 239.64264, ..., 239.64697, 239.64592,
         239.64497]],

       [[236.44669, 236.44669, 236.44669, ..., 236.39807, 236.41527,
         236.44107],
        [236.25679, 236.30174, 236.35736, ..., 236.13501, 236.17119,
         236.21344],
        [237.61913, 237.68161, 237.7445 , ..., 237.47157, 237.51117,
         237.56056],
        ...,
        [243.06339, 242.99643, 242.9374 , ..., 243.24942, 243.18777,
         243.12567],
        [242.39528, 242.38954, 242.3846 , ..., 242.41232, 242.40703,
   

In [15]:
GMT_abr_yr =  GMT_abr.copy()
CCF_abr_yr =  CCF_abr.copy()
LWP_abr_yr =  LWP_abr.copy()

for i in range(shape_time_abr//12):
    
    
    for var1 in GMT_abr:
        GMT_abr_yr[var1][i,:,:]  = np.nanmean(GMT_abr[var1][i*12:(i+1)*12, :, :], axis=0)
    for var2 in CCF_abr:
        CCF_abr_yr[var2][i,:,:]  = np.nanmean(CCF_abr[var2][i*12:(i+1)*12, :, :], axis=0)
    for var3 in LWP_abr:
        LWP_abr_yr[var3][i,:,:]  = np.nanmean(LWP_abr[var3][i*12:(i+1)*12, :, :], axis=0)


print(GMT_abr_yr['gmt'].shape, CCF_abr_yr['LTS'])   #..


'''
    Tg[i,:,:]     = np.nanmean(gmt[i*12:(i+1)*12,:,:], axis=0)
    LWP_yr[i,:,:]  = np.nanmean(LWP[i*12:(i+1)*12, latsi1:latsi0,:], axis=0)
    p_e_yr[i,:,:] = np.nanmean(MC[i*12:(i+1)*12, latsi1:latsi0,:], axis=0)
    SST_yr[i,:,:]  = np.nanmean(SST[i*12:(i+1)*12, latsi1:latsi0,:], axis=0)
    
    LTS_yr[i,:,:]  = np.nanmean(LTS_e[i*12:(i+1)*12, latsi1:latsi0,:], axis=0)
    SUB_yr[i,:,:]  = np.nanmean(Subsidence[i*12:(i+1)*12, latsi1:latsi0,:], axis=0)

    TWP_yr[i,:,:]  = np.nanmean(Twp[i*12:(i+1)*12, latsi1:latsi0, :], axis=0)
    IWP_yr[i,:,:]  = np.nanmean(Iwp[i*12:(i+1)*12, latsi1:latsi0, :], axis=0)
    PRW_yr[i,:,:]  = np.nanmean(prw_pi[i*12:(i+1)*12, latsi1:latsi0, :], axis=0)
'''

GMT_pi_yr =  GMT_pi.copy()
CCF_pi_yr =  CCF_pi.copy()
LWP_pi_yr =  LWP_pi.copy()

for i in range(shape_time_pi):     #..shape in 3yr !!
    
    for var1 in GMT_pi:
        GMT_pi_yr[var1][i,:,:]  = np.nanmean(GMT_pi[var1][i*12:(i+1)*12, :, :], axis=0)
    for var2 in CCF_abr:
        CCF_pi_yr[var2][i,:,:]  = np.nanmean(CCF_pi[var2][i*12:(i+1)*12, :, :], axis=0)
    for var3 in LWP_abr:
        LWP_pi_yr[var3][i,:,:]  = np.nanmean(LWP_pi[var3][i*12:(i+1)*12, :, :], axis=0)


/glade/u/ssg/ch/usr/jupyterhub/envs/npl-3.7.9/dav/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: Mean of empty slice
  if __name__ == '__main__':
/glade/u/ssg/ch/usr/jupyterhub/envs/npl-3.7.9/dav/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: Mean of empty slice
  # This is added back by InteractiveShellApp.init_path()
/glade/u/ssg/ch/usr/jupyterhub/envs/npl-3.7.9/dav/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: Mean of empty slice
  del sys.path[0]


(1800, 192, 288) [[[-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]
  ...
  [16.933544158935547 16.934406280517578 16.934711456298828 ...
   16.93233299255371 16.932289123535156 16.932828903198242]
  [16.88613510131836 16.885292053222656 16.884241104125977 ...
   16.887800216674805 16.887147903442383 16.886642456054688]
  [16.838497161865234 16.838390350341797 16.83828353881836 ...
   16.838905334472656 16.838760375976562 16.838619232177734]]

 [[-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]
  ...
  [-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]]

 [[-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]
  ...
  [-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]]

 ...

 [[-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]
  ...
  [-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]]

 [[-- -- -- ... -- -- --]
 

/glade/u/ssg/ch/usr/jupyterhub/envs/npl-3.7.9/dav/lib/python3.7/site-packages/ipykernel_launcher.py:40: RuntimeWarning: Mean of empty slice
/glade/u/ssg/ch/usr/jupyterhub/envs/npl-3.7.9/dav/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: Mean of empty slice
/glade/u/ssg/ch/usr/jupyterhub/envs/npl-3.7.9/dav/lib/python3.7/site-packages/ipykernel_launcher.py:44: RuntimeWarning: Mean of empty slice


In [ ]:
#..netCDF4 for retrieving variables in 'pi' and 'abr': 

#..abrupt4xCO2
sfc_T_abr       = read_var_mod(modn='CESM2', consort='NCAR', varnm='ts', cmip='cmip6', exper='abrupt-4xCO2', ensmem='r1i1p1f1', typevar='Amon', gg='gn', read_p=False, time1=[1, 1, 1], time2=[150, 12, 31])[0]

T_700_alevs_abr,P,lat_abr,lon_abr,times_abr = read_var_mod(modn='CESM2', consort='NCAR', varnm='ta', cmip='cmip6', exper='abrupt-4xCO2', ensmem='r1i1p1f1', typevar='Amon', gg='gn', read_p=True, time1=[1, 1, 1], time2=[150, 12, 31])
T_700_abr = T_700_alevs_abr[:, 3,:,:]   #..700 hPa levels

sfc_P_abr       = read_var_mod(modn='CESM2', consort='NCAR', varnm='ps', cmip='cmip6', exper='abrupt-4xCO2', ensmem='r1i1p1f1', typevar='Amon', gg='gn', read_p=False, time1=[1, 1, 1], time2=[150, 12, 31])[0]   

sub_abr         = read_var_mod(modn='CESM2', consort='NCAR', varnm='wap', cmip='cmip6', exper='abrupt-4xCO2', ensmem='r1i1p1f1', typevar='Amon', gg='gn', read_p=True, time1=[1, 1, 1], time2=[150, 12, 31])[0][:, 5,:,:] 
#..500mb downward motions

clivi_abr       = read_var_mod(modn='CESM2', consort='NCAR', varnm='clivi', cmip='cmip6', exper='abrupt-4xCO2', ensmem='r1i1p1f1', typevar='Amon', gg='gn', read_p=False, time1=[1, 1, 1], time2=[150, 12, 31])[0]
clwvi_abr       = read_var_mod(modn='CESM2', consort='NCAR', varnm='clwvi', cmip='cmip6', exper='abrupt-4xCO2', ensmem='r1i1p1f1', typevar='Amon', gg='gn', read_p=False, time1=[1, 1, 1], time2=[150, 12, 31])[0]
tas_abr         = read_var_mod(modn='CESM2', consort='NCAR', varnm='tas', cmip='cmip6', exper='abrupt-4xCO2', ensmem='r1i1p1f1', typevar='Amon', gg='gn', read_p=False, time1=[1, 1, 1], time2=[150, 12, 31])[0]

P_abr           = read_var_mod(modn='CESM2', consort='NCAR', varnm='pr', cmip='cmip6', exper='abrupt-4xCO2', ensmem='r1i1p1f1', typevar='Amon', gg='gn', read_p=False, time1=[1, 1, 1], time2=[150, 12, 31])[0]

E_abr           = read_var_mod(modn='CESM2', consort='NCAR', varnm='evspsbl', cmip='cmip6', exper='abrupt-4xCO2', ensmem='r1i1p1f1', typevar='Amon', gg='gn', read_p=False, time1=[1, 1, 1], time2=[150, 12, 31])[0]
print(sfc_T_abr.shape)

prw_abr         = read_var_mod(modn='CESM2', consort='NCAR', varnm='prw', cmip='cmip6', exper='abrupt-4xCO2', ensmem='r1i1p1f1', typevar='Amon', gg='gn', read_p=False, time1=[1, 1, 1], time2=[150, 12, 31])[0]
#..1800 months =150 yrs for CESM2:abrupt experiemnt



#..Total Water Path
#fn_wvp_abr  = xr.open_dataset('/glade/collections/cdg/data/CMIP6/CMIP/NCAR/CESM2/abrupt-4xCO2/r1i1p1f1/Amon/prw/gn/v20190828/prw_Amon_CESM2_abrupt-4xCO2_r1i1p1f1_gn_000101-015012.nc')

#prw_abr = np.array(fn_wvp_abr.variables['prw'])   #..Water Vapor Path , kg m-2

#fn_wvp_pi   = xr.open_dataset('/glade/collections/cdg/data/CMIP6/CMIP/NCAR/CESM2/piControl/r1i1p1f1/Amon/prw/gn/files/d20190320/prw_Amon_CESM2_piControl_r1i1p1f1_gn_000101-009912.nc')
#prw_pi  = np.array(fn_wvp_pi.variables['prw'])


#..pi-Control
sfc_T       = read_var_mod(modn='CESM2', consort='NCAR', varnm='ts', cmip='cmip6', exper='piControl', ensmem='r1i1p1f1', typevar='Amon', gg='gn', read_p=False, time1=[1, 1, 1], time2=[99, 12, 31])[0]

T_700_alevs,times_pi = read_var_mod(modn='CESM2', consort='NCAR', varnm='ta', cmip='cmip6', exper='piControl', ensmem='r1i1p1f1', typevar='Amon', gg='gn', read_p=True, time1=[1, 1, 1], time2=[99, 12, 31])[0],[4]
T_700       = T_700_alevs[:, 3,:,:]

sfc_P       = read_var_mod(modn='CESM2', consort='NCAR', varnm='ps', cmip='cmip6', exper='piControl', ensmem='r1i1p1f1', typevar='Amon', gg='gn', read_p=False, time1=[1, 1, 1], time2=[99, 12, 31])[0]   
#..sea surface Pressure, Units in Pa

sub         = read_var_mod(modn='CESM2', consort='NCAR', varnm='wap', cmip='cmip6', exper='piControl', ensmem='r1i1p1f1', typevar='Amon', gg='gn', read_p=True, time1=[1, 1, 1], time2=[99, 12, 31])[0][:, 5,:,:] 
#..500mb downward mot0001

clivi       = read_var_mod(modn='CESM2', consort='NCAR', varnm='clivi', cmip='cmip6', exper='piControl', ensmem='r1i1p1f1', typevar='Amon', gg='gn', read_p=False, time1=[1, 1, 1], time2=[99, 12, 31])[0]
#..ICE WATER PATH, Units in kg m^-2
clwvi       = read_var_mod(modn='CESM2', consort='NCAR', varnm='clwvi', cmip='cmip6', exper='piControl', ensmem='r1i1p1f1', typevar='Amon', gg='gn', read_p=False, time1=[1, 1, 1], time2=[99, 12, 31])[0]
tas         = read_var_mod(modn='CESM2', consort='NCAR', varnm='tas', cmip='cmip6', exper='piControl', ensmem='r1i1p1f1', typevar='Amon', gg='gn', read_p=False, time1=[1, 1, 1], time2=[99, 12, 31])[0]
#..2-m air Temperature, for 'gmt'

P           = read_var_mod(modn='CESM2', consort='NCAR', varnm='pr', cmip='cmip6', exper='piControl', ensmem='r1i1p1f1', typevar='Amon', gg='gn', read_p=False, time1=[1, 1, 1], time2=[99, 12, 31])[0]
#..Precipitation, Units in kg m^-2 s^-1 = mm *s^-1
E           = read_var_mod(modn='CESM2', consort='NCAR', varnm='evspsbl', cmip='cmip6', exper='piControl', ensmem='r1i1p1f1', typevar='Amon', gg='gn', read_p=False, time1=[1, 1, 1], time2=[99, 12, 31])[0]
#..Evaporations, Units also in kg m^-2 s^-1 = mm *s^-1

print(sfc_T.shape)


prw_pi      =read_var_mod(modn='CESM2', consort='NCAR', varnm='prw', cmip='cmip6', exper='piControl', ensmem='r1i1p1f1', typevar='Amon', gg='gn', read_p=False, time1=[1, 1, 1], time2=[99, 12, 31])[0]